In [ ]:
# Configure matplotlib.
%matplotlib inline

In [ ]:
# Import our package.
import sys, importlib
sys.path.append("/home/ubuntu/cell_counting")

from src import dataset, visualization, preprocess, metric, losses
from src.model import model
from src.model import neural_net
from src.model.segmentation.convnet1 import convnet1
from IPython import display

In [ ]:
# (if changes are made) Re-import our package.
for module in (dataset, visualization, preprocess, metric, model, neural_net, convnet1, losses):
    importlib.reload(module)

In [ ]:
# Load the dataset, processing it as a collection of image-mask pairs.
images_masks = dataset.Dataset(1)
images_masks.load_image_mask_pairs("/home/ubuntu/cell_counting/data/easy/raw/images",
                                   "/home/ubuntu/cell_counting/data/easy/raw/masks", (2448*2, 2448*2))

In [ ]:
# Plot a batch.
inputs, outputs = images_masks.get_batch(3)
visualization.show_image_grid(inputs, 1, 3, 3, 10, "images")
visualization.show_image_grid(outputs, 1, 3, 3, 10, "masks")

In [ ]:
# Normalize the images.
#def normalize(batch):
#    inputs, outputs = batch
#    inputs = preprocess.smdm_normalize(inputs, 61, "REFLECT")
#    return (inputs, outputs)
#images_masks.map_batch(normalize)

In [ ]:
# Plot a batch.
#inputs, outputs = images_masks.get_batch(3)
#visualization.show_image_grid(inputs, 1, 3, 2, 6, "images")
#visualization.show_image_grid(outputs, 1, 3, 2, 6, "masks")

In [ ]:
# Extract patches from the images.
def extract_patches(example):
    input_, output = example
    input_ = preprocess.extract_patches(input_, 61, 1000)
    output = preprocess.extract_patches(output, 61, 1000)
    examples = [(input_[i, ...] / 255, 0 if all(output[i, 61//2 + 1, 61//2 + 1] > 200) else 1) for i in range(input_.shape[0])]
    return examples
images_masks.map(extract_patches)
images_masks.set_segment_size(10)

In [ ]:
images_masks._segment_dir

In [ ]:
# Plot a batch.
inputs, outputs = images_masks.get_batch(4*8)
visualization.show_image_grid(inputs * 255, 4, 8, 2.5*4, 16, "images",
                              [("colony" if outputs[i] == 1 else "not colony") for i in range(outputs.shape[0])])

In [ ]:
# Split the dataset.
train, test = images_masks.split(0.4)

In [ ]:
# Create the net.
import tensorflow as tf
net = convnet1.ConvNet1("saves/18-01-16-PM-09-07", 120, train.size())

In [ ]:
# Create some metrics.
train_data = train.get_batch(1000)
test_data = test.get_batch(1000)
def loss_fn(predicted, actual):
    loss = tf.losses.softmax_cross_entropy(tf.one_hot(tf.cast(actual, tf.int32), 3), predicted)
    with tf.Session().as_default():
        return loss.eval()
metrics = {
    "train_loss": metric.LossMetric(train_data, loss_fn),
    "test_loss": metric.LossMetric(test_data, loss_fn),
    "pred_thpt": metric.PredictionThroughputMetric(test_data)
}

In [ ]:
# Make a function for plotting the metrics.
def plot_metrics():
    xs, ys = metrics["train_loss"].get_results()
    visualization.plot_line(xs, ys, "Training Loss", "training examples seen", "cross-entropy loss on training data",
                            3, 10)
    xs, ys = metrics["test_loss"].get_results()
    visualization.plot_line(xs, ys, "Test Loss", "training examples seen", "cross-entropy loss on test data", 3, 10)
    xs, ys = metrics["pred_thpt"].get_results()
    visualization.plot_line(xs, ys, "Training Throughput", "training examples seen", "speed of training in examples/s",
                            3, 10)

In [ ]:
# Alternately train and evaluate the net for 20 minutes.
for _ in range(10):
    net.train(train, 3*60)
    net.evaluate(metrics)
    display.clear_output()
    plot_metrics()

In [ ]:
# Alternately train and evaluate the net for 20 minutes.
for _ in range(10):
    net.train(train, 3*60)
    net.evaluate(metrics)
    display.clear_output()
    plot_metrics()

In [ ]:
# Close the dataset.
microbia_segments.close()